## HW3 - Dynamic Programming and Uncertainty (Due 8/12)

In this final homework assignment you will be asked to solve 1 IP and 2 dynamic programming problems. The first DP will focus on a completely static offline problem. In the second, you may use both dynamic programming and some of the ideas we learned about in our 1-lecture simulation unit.

In [1]:
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt

#### Problem 1: Optimal Shift Planning

In the planning of monthly production for the next months a company must, in each month, operate either a normal shift or an extended shift (if it produces at all - it can also "not produce"). We will aim to minimize the company's payroll costs while satisfying all production requirements. 

The payroll costs of a normal shift are (all costs are in USD) 100,000 per month, and in a month that is run as a normal shift up to 6,000 units can be produced. An extended shift month costs 180,000 and allows for production of between 6,000 and 11,000 units per month. Production constraints are such that if the company produces anything at all in a particular month it must produce at least 2,000 units.

A month with no production happening costs 20,000. Note here that, for either type of shift, the cost incurred is fixed by a union guarantee agreement and so is independent of the amount produced.

However, the company also incurs an cost of 100,000 each time it changes from a normal shift or a no-production shift in one month to an extended shift in the next month. No extra cost is incurred in changing the production schedule between any other types of shifts (e.g., from normal month to no production incurs 0 cost, same for going from an extended shift in one month to a normal shift in the next). This extra cost is also not inccurred when remaining in extended shift for several consecutive months.

The cost of holding stock is $20 per unit per month and is based on the stock held at the end of each month, after demand for that month has been satisfied. The initial stock is 3,000 units. However, the amount in stock at the end of the last month should also be at least 2,000 units. In the most recent month (relevant for cost of switching to extended shift) the company operated in a normal shift. The demand for the company's product in each of the months is known to be as provided in the list "demand" below.

The company wants a production plan for the next months that avoids stockouts (i.e., meets all demand), and minimizes the combined costs that are described above. Formulate and solve this problem as an integer program (optimal solution should come out as 1.82M).

Hint: it may pay off to have up to 6 kinds of decision variables for each month: three binaries that signal the type of production (no production, normal, extended), one continuous variable for the production quantities, another continuous one for the leftover inventory at the end of a month (both of these may also be integer), and binary variables that signal whether extended production occurs in one month after normal/no production in the previous.

In [3]:
demand = [3000, 4000, 9000,0, 7000, 15000, 5000, 10000, 5000, 11000, 2000]
# You may or may not include the 0 in the last month; if you do, then appending 
# another may make your code run smoother in allowing you to use demand[-1] to 
# access a month that is not part of the question
demand.append(0)
num_months = len(demand)-1

normal_cost = 100000
normal_production = 6000
minimum_production = 2000

extended_cost = 180000
extended_production = 11000

no_production_cost = 20000

cost_of_changing_to_extended = 100000

holding_cost_per_unit = 20
initial_stock = 3000
end_inventory = 2000

m = gp.Model("model")

#dec var
pro_quantity = {}
inventory = {}
extended = {}
normal = {}
no_shift = {}
change_to_extended = {}


inventory[0] = initial_stock
extended[0] = 0

for month in range(num_months):
    pro_quantity[month+1] = m.addVar(lb=0, vtype=gp.GRB.INTEGER)
    inventory[month+1] = m.addVar(lb=0, vtype=gp.GRB.INTEGER)
    extended[month+1] = m.addVar(vtype= gp.GRB.BINARY)
    normal[month+1] = m.addVar(vtype= gp.GRB.BINARY)
    no_shift[month+1] = m.addVar(vtype= gp.GRB.BINARY)
    change_to_extended[month+1] = m.addVar(vtype= gp.GRB.BINARY)
    
    
for month in range(num_months):
    # we need to have one of the 3 kinds of shifts
    m.addConstr(extended[month+1] + normal[month+1] + no_shift[month+1] == 1)
    m.addConstr(inventory[month+1] >= demand[month])
    # "Flow constraints" that capture the amount of inventory given how much has been
    # produced, how much we had, and how much was used up by demand
    m.addConstr(inventory[month+1] == inventory[month]-demand[month-1] + pro_quantity[month+1])
    
    ####what kind of shift can we be in in given how much we produce###
    # if we produce, we produce at least minimum_production
    m.addConstr(pro_quantity[month+1] >= minimum_production*(normal[month+1] +extended[month+1]))
                
    # we cannot produce more than allowed for a given shift
    m.addConstr(pro_quantity[month+1] <= normal_production*normal[month+1] 
                + extended_production*extended[month+1])
    # if we changed to extended, we pay the cost
    m.addConstr(extended[month+1] - extended[month] 
                <= change_to_extended[month+1])
    
#The amount of stock at the end of the last month should also be at least end_inventory.
m.addConstr(inventory[num_months]-demand[num_months-1] >= end_inventory)
    
production_cost = sum(normal_cost*normal[month+1] 
                      + extended_cost*extended[month+1] 
                      + no_production_cost*no_shift[month+1] 
                      + cost_of_changing_to_extended*change_to_extended[month+1]
                      + holding_cost_per_unit*(inventory[month+1]-demand[month]) for month in range(num_months))

m.setObjective(production_cost, gp.GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 67 rows, 66 columns and 175 nonzeros
Model fingerprint: 0xbf977a56
Variable types: 0 continuous, 66 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective 2140000.0000
Presolve removed 25 rows and 25 columns
Presolve time: 0.00s
Presolved: 42 rows, 41 columns, 131 nonzeros
Found heuristic solution: objective 1999960.0000
Variable types: 0 continuous, 41 integer (30 binary)

Root relaxation: objective 1.585545e+06, 24 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1585545.45    0   12 1999960.00 1585545.45  20.7%     -    0s
H    0     0                    1839980.0000 1585545.45  13.8%     -    0s
H    0     0  

In [10]:
for month in range(num_months):
    print(extended[month+1].x,pro_quantity[month+1].x,(inventory[month+1].x-demand[month])*holding_cost_per_unit, pro_quantity[month+1].x)

-0.0 2000.0 40000.0 2000.0
-0.0 5000.0 60000.0 5000.0
-0.0 6000.0 0.0 6000.0
-0.0 0.0 0.0 0.0
1.0 11000.0 80000.0 11000.0
1.0 11000.0 0.0 11000.0
1.0 5000.0 0.0 5000.0
1.0 10000.0 0.0 10000.0
1.0 5000.0 0.0 5000.0
1.0 11000.0 0.0 11000.0
0.0 4000.0 40000.0 4000.0


#### Bonus question: 
What changes if you add the constraint that in an extended month you need to produce at least 6000 units? Can you explain the operational implication/underlying tradeoffs?

Somewhat surprisingly it can be cost-efficient to be in extended production mode even when not producing more than 6000 units; the reason is the relatively high cost (100,000) of switching to extended production mode. Instead, it can pay off to just stay in extended production mode.

#### Problem 2: Checkerboard

Consider the following problem. You are at the bottom-left corner of a chessboard ("A1"). Each field on the board has a value. The value of a field is stored in the dictionary "values" defined in the cell below. For each field you visit, you collect the value of that field; hwowever, you are only allowed to move "up" or "right" (e.g., from "A1" to "B1" or from "A1" to "A2"). Find the maximum value you can collect as you move from "A1" eventually to "H8"? What sequence of fields to visit will yield that value?

(An optimal solution should yield 1111 in value)

In [3]:
values = {('A', 1): 87, ('A', 2): 6, ('A', 3): 81, ('A', 4): 89, ('A', 5): 99, ('A', 6): 8, ('A', 7): 47,
 ('A', 8): 81, ('B', 1): 18, ('B', 2): 17, ('B', 3): 50, ('B', 4): 66, ('B', 5): 91, ('B', 6): 47, ('B', 7): 98,
 ('B', 8): 55, ('C', 1): 21, ('C', 2): 1, ('C', 3): 16, ('C', 4): 9, ('C', 5): 70, ('C', 6): 96, ('C', 7): 93,
 ('C', 8): 34, ('D', 1): 19, ('D', 2): 63, ('D', 3): 52, ('D', 4): 74, ('D', 5): 50, ('D', 6): 5, ('D', 7): 90,
 ('D', 8): 45, ('E', 1): 35, ('E', 2): 3, ('E', 3): 30, ('E', 4): 43, ('E', 5): 62, ('E', 6): 57, ('E', 7): 20,
 ('E', 8): 54, ('F', 1): 51, ('F', 2): 70, ('F', 3): 70, ('F', 4): 49, ('F', 5): 20, ('F', 6): 8, ('F', 7): 5,
 ('F', 8): 92, ('G', 1): 70, ('G', 2): 79, ('G', 3): 21, ('G', 4): 86, ('G', 5): 42, ('G', 6): 76, ('G', 7): 67,
 ('G', 8): 33, ('H', 1): 1, ('H', 2): 26, ('H', 3): 78, ('H', 4): 53, ('H', 5): 39, ('H', 6): 73, ('H', 7): 24,
 ('H', 8): 85}

Hint: When at field "H8" you get a value of 85, but will not receive any more from moving. At field "H7" you receive 24, and you will have to move to "H8" next getting you another 85. At "G7" you get 67, and you get the better of moving to "G8" (33) and then to "H8" or to "H7" (24) and then to "H8". How much value will you collect in the remaining path starting from "G7"?

In [4]:
next_letter = {'A':['A','B'], 'B': ['B','C'], 'C':['C','D'], 'D':['D', 'E'], 'E':['E', 'F'], 'F':['F', 'G'], 
               'G':['G', 'H'], 'H':['H']}
next_number = {1:[1,2], 2:[2,3], 3:[3,4], 4:[4,5], 5:[5,6], 6:[6,7], 7:[7,8], 8: [8]}

In [5]:
next_step = {}
for x in values:
    next_step[x]=[]
    letter, number = x
    for l in next_letter[letter]:
        for n in next_number[number]:
            if (l==letter or n==number) and not (l==letter and n==number): next_step[x].append((l,n))

In [6]:
remaining_value = {}
remaining_value[('H',8)] = values[('H', 8)]
best_path = {}
for i in [8, 7, 6, 5, 4, 3, 2, 1]:
    for j in ['H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']:
        best_step, opt = None, 0
        for x in next_step[(j,i)]:
            if remaining_value[x] >= opt: best_step, opt = x,remaining_value[x]
        best_path[(j,i)] = best_step
        remaining_value[(j,i)]=opt + values[(j,i)]

In [7]:
x = ('A',1)
while x!=('H',8):
    print(x, values[x], remaining_value[x], best_path[x])
    x=best_path[x]
print(x, values[x], remaining_value[x])

('A', 1) 87 1111 ('A', 2)
('A', 2) 6 1024 ('A', 3)
('A', 3) 81 1018 ('A', 4)
('A', 4) 89 937 ('A', 5)
('A', 5) 99 848 ('B', 5)
('B', 5) 91 749 ('C', 5)
('C', 5) 70 658 ('C', 6)
('C', 6) 96 588 ('C', 7)
('C', 7) 93 492 ('D', 7)
('D', 7) 90 399 ('D', 8)
('D', 8) 45 309 ('E', 8)
('E', 8) 54 264 ('F', 8)
('F', 8) 92 210 ('G', 8)
('G', 8) 33 118 ('H', 8)
('H', 8) 85 85


#### Problem 3: Secretary problem (implement solution from class)

In class we saw the secretary problem, in which the CEO's goal was to maximize the expected score of the candidate. We saw the intuition then of what a solution should look like: she should accept in round $t$ if (in expectation) the remaining $t-1$ candidates will not be better. Use dynamic programming and simulation to identify the threshold for accepting with $t$ candidates remaining for $t=1,2,3,4,5$.

Hint: the cell below creates 10,000 samples of 5 random secretary arrivals. Use those to estimate (for a given threshold) the expected quality of the last arrival. You'd want to accept the second-to-last if he is better than that expected quality. You can also use these arrivals to estimate the probability that the second-to-last arrival would be better than the expected quality, and use those 2 together to determine the expected quality the CEO will get out of the last 2 combined, which you can then use for the third-to-last (and so forth)...

In [8]:
arrivals = {}
for k in range(10**4):
    arrivals[k] = np.random.randint(0,101,5)

In [9]:
expected_score = {0:0}

In [10]:
for t in range(1,5):
    score_at_t = np.mean([x[-t] for x in arrivals.values() if x[-t]>expected_score[t-1]])
    prob_of_accepting_t = np.mean([x[-t]>=expected_score[t-1] for  x in arrivals.values()])
    expected_score[t] = prob_of_accepting_t*score_at_t + (1-prob_of_accepting_t) * expected_score[t-1]
    print(prob_of_accepting_t)
expected_score

1.0
0.4898
0.3723
0.306


{0: 0,
 1: 50.533239038189535,
 2: 62.64375855728431,
 3: 69.70698724640737,
 4: 74.40744914900671}

Next, repeat the above, but rather than assuming that the quality score of each candidate is uniform between 0 and 100, assume it is normally distributed with mean 500 and standard deviation 20 (if you did the above right, this should only require a change in one line).

In [11]:
arrivals = {}
for k in range(100000):
    arrivals[k] = np.random.normal(500,20,7)
expected_score = {0:0}
for t in range(1,7):
    score_at_t = np.mean([x[-t] for x in arrivals.values() if x[-t]>=expected_score[t-1]])
    prob_of_accepting_t = np.mean([x[-t]>expected_score[t-1] for  x in arrivals.values()])
    expected_score[t] = prob_of_accepting_t*score_at_t + (1-prob_of_accepting_t) * expected_score[t-1]
    print(prob_of_accepting_t)
expected_score

1.0
0.50053
0.34654
0.2637
0.2138
0.1801


{0: 0,
 1: 500.01217354856124,
 2: 507.97088038098303,
 3: 512.6249683531737,
 4: 515.8108979960259,
 5: 518.2326436694888,
 6: 520.1746429537388}